# A full business solution

## Now we will take our project from Day 1 to the next level

### BUSINESS CHALLENGE:

Create a product that builds a Brochure for a company to be used for prospective clients, investors and potential recruits.

We will be provided a company name and their primary website.

See the end of this notebook for examples of real-world business applications.

And remember: I'm always available if you have problems or ideas! Please do reach out.

In [1]:
# imports
# If these fail, please check you're running from an 'activated' environment with (llms) in the command prompt

import os
import requests
import json
from typing import List
from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display, update_display
from openai import OpenAI

In [2]:
# Initialize and constants

load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key)>10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key? Please visit the troubleshooting notebook!")
    
MODEL = 'gpt-4o-mini'
openai = OpenAI()

API key looks good so far


In [3]:
# A class to represent a Webpage

# Some websites need you to use proper headers when fetching them:
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}

class Website:
    """
    A utility class to represent a Website that we have scraped, now with links
    """

    def __init__(self, url):
        self.url = url
        response = requests.get(url, headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        if soup.body:
            for irrelevant in soup.body(["script", "style", "img", "input"]):
                irrelevant.decompose()
            self.text = soup.body.get_text(separator="\n", strip=True)
        else:
            self.text = ""
        links = [link.get('href') for link in soup.find_all('a')]
        self.links = [link for link in links if link]

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [4]:
ed = Website("https://edwarddonner.com")
print(ed.get_contents())
ed.links

Webpage Title:
Home - Edward Donner
Webpage Contents:
Home
Connect Four
Outsmart
An arena that pits LLMs against each other in a battle of diplomacy and deviousness
About
Posts
Well, hi there.
I’m Ed. I like writing code and experimenting with LLMs, and hopefully you’re here because you do too. I also enjoy DJing (but I’m badly out of practice), amateur electronic music production (
very
amateur) and losing myself in
Hacker News
, nodding my head sagely to things I only half understand.
I’m the co-founder and CTO of
Nebula.io
. We’re applying AI to a field where it can make a massive, positive impact: helping people discover their potential and pursue their reason for being. Recruiters use our product today to source, understand, engage and manage talent. I’m previously the founder and CEO of AI startup untapt,
acquired in 2021
.
We work with groundbreaking, proprietary LLMs verticalized for talent, we’ve
patented
our matching model, and our award-winning platform has happy customers a

['https://edwarddonner.com/',
 'https://edwarddonner.com/connect-four/',
 'https://edwarddonner.com/outsmart/',
 'https://edwarddonner.com/about-me-and-about-nebula/',
 'https://edwarddonner.com/posts/',
 'https://edwarddonner.com/',
 'https://news.ycombinator.com',
 'https://nebula.io/?utm_source=ed&utm_medium=referral',
 'https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html',
 'https://patents.google.com/patent/US20210049536A1/',
 'https://www.linkedin.com/in/eddonner/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/12/21/llm-resources-superdatascience/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'https://edwarddonner.com/2024/11/13/llm-engineering-resources/',
 'ht

## First step: Have GPT-4o-mini figure out which links are relevant

### Use a call to gpt-4o-mini to read the links on a webpage, and respond in structured JSON.  
It should decide which links are relevant, and replace relative links such as "/about" with "https://company.com/about".  
We will use "one shot prompting" in which we provide an example of how it should respond in the prompt.

This is an excellent use case for an LLM, because it requires nuanced understanding. Imagine trying to code this without LLMs by parsing and analyzing the webpage - it would be very hard!

Sidenote: there is a more advanced technique called "Structured Outputs" in which we require the model to respond according to a spec. We cover this technique in Week 8 during our autonomous Agentic AI project.

In [5]:
link_system_prompt = "You are provided with a list of links found on a webpage. \
You are able to decide which of the links would be most relevant to include in a brochure about the company, \
such as links to an About page, or a Company page, or Careers/Jobs pages.\n"
link_system_prompt += "You should respond in JSON as in this example:"
link_system_prompt += """
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}
"""

In [6]:
print(link_system_prompt)

You are provided with a list of links found on a webpage. You are able to decide which of the links would be most relevant to include in a brochure about the company, such as links to an About page, or a Company page, or Careers/Jobs pages.
You should respond in JSON as in this example:
{
    "links": [
        {"type": "about page", "url": "https://full.url/goes/here/about"},
        {"type": "careers page": "url": "https://another.full.url/careers"}
    ]
}



In [7]:
def get_links_user_prompt(website):
    user_prompt = f"Here is the list of links on the website of {website.url} - "
    user_prompt += "please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. \
Do not include Terms of Service, Privacy, email links.\n"
    user_prompt += "Links (some might be relative links):\n"
    user_prompt += "\n".join(website.links)
    return user_prompt

In [8]:
print(get_links_user_prompt(ed))

Here is the list of links on the website of https://edwarddonner.com - please decide which of these are relevant web links for a brochure about the company, respond with the full https URL in JSON format. Do not include Terms of Service, Privacy, email links.
Links (some might be relative links):
https://edwarddonner.com/
https://edwarddonner.com/connect-four/
https://edwarddonner.com/outsmart/
https://edwarddonner.com/about-me-and-about-nebula/
https://edwarddonner.com/posts/
https://edwarddonner.com/
https://news.ycombinator.com
https://nebula.io/?utm_source=ed&utm_medium=referral
https://www.prnewswire.com/news-releases/wynden-stark-group-acquires-nyc-venture-backed-tech-startup-untapt-301269512.html
https://patents.google.com/patent/US20210049536A1/
https://www.linkedin.com/in/eddonner/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2025/01/23/llm-workshop-hands-on-with-agents-resources/
https://edwarddonner.com/2024/12/21/

In [9]:
def get_links(url):
    website = Website(url)
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": link_system_prompt},
            {"role": "user", "content": get_links_user_prompt(website)}
      ],
        response_format={"type": "json_object"}
    )
    result = response.choices[0].message.content
    return json.loads(result)

In [10]:
get_links("https://anthropic.com")

{'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'},
  {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'},
  {'type': 'news page', 'url': 'https://www.anthropic.com/news'},
  {'type': 'research page', 'url': 'https://www.anthropic.com/research'}]}

In [ ]:
# Anthropic has made their site harder to scrape, so I'm using HuggingFace..

huggingface = Website("https://huggingface.co")
huggingface.links

In [ ]:
get_links("https://huggingface.co")

## Second step: make the brochure!

Assemble all the details into another prompt to GPT4-o

In [11]:
def get_all_details(url):
    result = "Landing page:\n"
    result += Website(url).get_contents()
    links = get_links(url)
    print("Found links:", links)
    for link in links["links"]:
        result += f"\n\n{link['type']}\n"
        result += Website(link["url"]).get_contents()
    return result

In [12]:
print(get_all_details("https://anthropic.com"))

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}
Landing page:
Webpage Title:
Home \ Anthropic
Webpage Contents:
Skip to main content
Skip to footer
Claude
API
Research
Commitments
Learn
News
Try Claude
AI
research
and
products
that put safety at the frontier
Claude.ai
Meet Claude 3.7 Sonnet
Claude 3.7 Sonnet, our most intelligent AI model, is now available.
Talk to Claude
API
Build with Claude
Create AI-powered applications and custom experiences using Claude.
Learn more
Claude 3.7 Sonnet and Claude Code
Introducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding.
Read the anno

In [13]:
system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
Include details of company culture, customers and careers/jobs if you have the information."

# Or uncomment the lines below for a more humorous brochure - this demonstrates how easy it is to incorporate 'tone':

# system_prompt = "You are an assistant that analyzes the contents of several relevant pages from a company website \
# and creates a short humorous, entertaining, jokey brochure about the company for prospective customers, investors and recruits. Respond in markdown.\
# Include details of company culture, customers and careers/jobs if you have the information."


In [14]:
def get_brochure_user_prompt(company_name, url):
    user_prompt = f"You are looking at a company called: {company_name}\n"
    user_prompt += f"Here are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\n"
    user_prompt += get_all_details(url)
    user_prompt = user_prompt[:10_000] # Truncate if more than 5,000 characters
    return user_prompt

In [16]:
get_brochure_user_prompt("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


'You are looking at a company called: Anthropic\nHere are the contents of its landing page and other relevant pages; use this information to build a short brochure of the company in markdown.\nLanding page:\nWebpage Title:\nHome \\ Anthropic\nWebpage Contents:\nSkip to main content\nSkip to footer\nClaude\nAPI\nResearch\nCommitments\nLearn\nNews\nTry Claude\nAI\nresearch\nand\nproducts\nthat put safety at the frontier\nClaude.ai\nMeet Claude 3.7 Sonnet\nClaude 3.7 Sonnet, our most intelligent AI model, is now available.\nTalk to Claude\nAPI\nBuild with Claude\nCreate AI-powered applications and custom experiences using Claude.\nLearn more\nClaude 3.7 Sonnet and Claude Code\nIntroducing Claude 3.7 Sonnet, our most intelligent model yet and the first hybrid reasoning model. We’re also launching Claude Code, an agentic tool for coding.\nRead the announcement\nModel details\nClaude 3.7 Sonnet\nResearch insights\nClaude\'s extended thinking\nOur Work\nProduct\nClaude for Enterprise\nSep 4, 

In [17]:
def create_brochure(company_name, url):
    response = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
    )
    result = response.choices[0].message.content
    display(Markdown(result))

In [18]:
create_brochure("Anthropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'team page', 'url': 'https://www.anthropic.com/team'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}, {'type': 'solutions page', 'url': 'https://www.anthropic.com/solutions/coding'}]}


# Anthropic Brochure

---

## 📢 Welcome to Anthropic

**At Anthropic, we are committed to building AI systems that prioritize safety, reliability, and interpretability.** With our headquarters in San Francisco, our interdisciplinary team works meticulously on research and product development to address the opportunities and risks associated with AI, ensuring it becomes a positive force for humanity.

---

## 🔍 Our Mission & Purpose

We believe that AI will profoundly impact the world, and our primary focus is to create systems that people can rely on. Our mission includes:
- Building reliable, interpretable, and steerable AI systems.
- Treating AI safety as a systematic science.
- Collaborating with stakeholders from various sectors to promote a culture of safety in AI development.

---

## 🤝 Company Culture

### Our Values
We abide by a set of core values that shape our interactions and decision-making:
1. **Act for the global good:** We strive for decisions that maximize positive outcomes for humanity.
2. **Hold light and shade:** We recognize the dual potential of AI for both risks and benefits.
3. **Be good to our users:** We prioritize kindness and generosity in all interactions.
4. **Ignite a race to the top on safety:** We promote the highest standards in AI safety and security.
5. **Do the simple thing that works:** We favor effective, straightforward solutions to challenges.
6. **Be helpful, honest, and harmless:** High trust and low ego guide our organization.
7. **Put the mission first:** Our shared purpose drives collaboration and expedites decision-making.

### Work Environment
Our team is made up of researchers, engineers, policy experts, and operators from diverse backgrounds. This rich tapestry of experiences—from academia to the tech industry—underpins our collaborative spirit and innovative approach to AI challenges. We also care deeply about our workspace, reflected in our love for plants and sustainability.

---

## 🌍 Who We Serve

**Our Products: Claude**
- **Claude 3.7 Sonnet**: Our most advanced AI model, designed for intelligent conversation and task performance.
- **Claude Code**: An agentic tool tailored for coding tasks, streamlining workflows for developers.

We cater to a diverse clientele, including businesses, non-profit organizations, and civil society groups. Our AI-powered applications are designed to enhance operational efficiency and impact.

---

## 🌟 Careers at Anthropic

We are looking for passionate individuals to join our mission-driven team. With open roles in various domains, including engineering, research, and operations, we invite you to explore career opportunities where you can make a meaningful impact.

### Why Join Us?
- **Health & Wellness**: Comprehensive health, dental, and vision insurance for you and your family.
- **Collaborative Environment**: Work alongside experts across various fields, contributing to a shared mission.
- **Impactful Work**: Be part of the frontier of AI research and development, shaping the future positively.

---

## 🚀 Join Us in Shaping the Future of AI

**Are you ready to help us build the future of safe AI?** At Anthropic, every team member plays a key role in our mission. Together, we are navigating the complexities of AI technology, aiming for a world where AI benefits all of humanity.

**Explore our open roles and apply today on our [Careers Page](#)**. 

---

📩 **Contact Us**: For more information about our products and research, visit [anthropic.com](https://www.anthropic.com) or reach out to our support team.

---

**Together, let's build a safer AI future!**

## Finally - a minor improvement

With a small adjustment, we can change this so that the results stream back from OpenAI,
with the familiar typewriter animation

In [19]:
def stream_brochure(company_name, url):
    stream = openai.chat.completions.create(
        model=MODEL,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": get_brochure_user_prompt(company_name, url)}
          ],
        stream=True
    )
    
    response = ""
    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        response = response.replace("```","").replace("markdown", "")
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
stream_brochure("Antropic", "https://anthropic.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://www.anthropic.com/company'}, {'type': 'careers page', 'url': 'https://www.anthropic.com/careers'}, {'type': 'research page', 'url': 'https://www.anthropic.com/research'}, {'type': 'news page', 'url': 'https://www.anthropic.com/news'}]}


# Anthropic Brochure

---

## About Us

**Anthropic** is an AI safety and research company based in San Francisco, dedicated to building reliable, interpretable, and steerable AI systems. We believe in the transformative power of AI and the responsibility that comes with it. Our interdisciplinary team comprises researchers, engineers, policy experts, and operational leaders who work collaboratively to generate research and develop cutting-edge AI products.

**Our Mission:** To ensure AI serves as a positive force, benefiting humanity while minimizing risks and harms.

---

## Our Products

### Claude
Meet **Claude 3.7 Sonnet**, our most intelligent AI model yet. Designed to power AI applications and enhance user experiences, Claude stands out as a hybrid reasoning model and is equipped with an agentic tool for coding—**Claude Code**.

#### Key Features:
- **Reliability and Interpretability:** AI systems that you can trust.
- **Tailored Solutions:** Designed for businesses, nonprofits, and civil society groups.
- **Developer-Friendly:** Comprehensive API for building custom applications.

---

## Our Research

At Anthropic, safety is at the forefront of our work. We conduct frontier research, exploring various modalities and developing novel safety techniques. Our approach is systematic; we apply our findings to enhance our products and share insights across the industry.

---

## Our Commitment to Safety

1. **Global Good:** We make decisions that benefit humanity and strive to guide the technological revolution positively.
2. **Transparent Practices:** Committed to responsible scaling, security, and compliance in AI development.
3. **Engaged Community:** Collaborating with diverse stakeholders, including governments, nonprofits, and academia.

---

## Company Culture

Our culture revolves around values that prioritize kindness, collaboration, and commitment to our mission. We operate as a **Public Benefit Corporation**, ensuring our endeavors align with long-term benefits for humanity.

### Core Values:
- Act for global good
- Hold light and shade
- Be good to our users
- Ignite a race to the top on safety
- Do the simple thing that works
- Be helpful, honest, and harmless
- Put the mission first

---

## Careers at Anthropic

Join us in building the future of safe AI! At Anthropic, we value diverse backgrounds and perspectives, offering a cohesive environment where each team member can thrive.

### What We Offer:
- **Health & Wellness:** Comprehensive health, dental, and vision insurance.
- **Career Growth:** Opportunities to advance within an interdisciplinary team.
- **Collaborative Environment:** Work alongside passionate individuals committed to AI safety.

**Want to join the team?** [Explore open roles](#)

---

## Get in Touch

For more information about our products, research, or career opportunities, visit our website at [Anthropic.ai](https://www.anthropic.ai) or contact us directly. 

Together, we can forge a future where AI empowers humanity safely and ethically.

--- 

*© 2025 Anthropic PBC*

In [23]:
stream_brochure("HiggingFace", "https://huggingface.co")

Found links: {'links': [{'type': 'about page', 'url': 'https://huggingface.co/huggingface'}, {'type': 'careers page', 'url': 'https://apply.workable.com/huggingface/'}, {'type': 'enterprise page', 'url': 'https://huggingface.co/enterprise'}, {'type': 'models page', 'url': 'https://huggingface.co/models'}, {'type': 'datasets page', 'url': 'https://huggingface.co/datasets'}, {'type': 'spaces page', 'url': 'https://huggingface.co/spaces'}, {'type': 'blog page', 'url': 'https://huggingface.co/blog'}, {'type': 'company page', 'url': 'https://huggingface.co/brand'}, {'type': 'community page', 'url': 'https://discuss.huggingface.co'}, {'type': 'GitHub page', 'url': 'https://github.com/huggingface'}, {'type': 'Twitter page', 'url': 'https://twitter.com/huggingface'}, {'type': 'LinkedIn page', 'url': 'https://www.linkedin.com/company/huggingface/'}]}


# **Hugging Face: The Future of AI**

---

## **About Us**
At **Hugging Face**, we are dedicated to building the future of AI through collaboration in the machine learning community. Our platform facilitates the sharing and deployment of **1M+ models**, **250k+ datasets**, and a diverse array of AI applications. Our mission is to democratize machine learning, making it accessible to everyone.

---

## **What We Offer**
- **Models & Datasets**: Explore and contribute to an expansive library of models and datasets tailored for every ML task.
- **Spaces**: Create and share innovative applications running on our platform.
- **Enterprise Solutions**: Advanced tools with top-tier security, analytics, and dedicated support to help organizations scale their AI capabilities.

### **Trending Models**
1. **sesame/csm-1b**
2. **google/gemma-3-27b-it**
3. **Qwen/QwQ-32B** 
4. **deepseek-ai/DeepSeek-R1**

---

## **Our Community**
Join over **50,000 organizations**, including major names like **Google**, **Microsoft**, **Amazon**, and **Intel**, leveraging our tools for cutting-edge AI projects. 

We pride ourselves on inclusivity and fostering a supportive environment where the **AI community can thrive**.

---

## **Company Culture**
We focus on collaboration, creativity, and innovation. Our diverse team of **208 members** is united by a common goal: to push the boundaries of what is possible with machine learning.

**Values:**
- Collaboration over Competition
- Open-Source contributions
- Continuous Learning and Improvement

---

## **Careers at Hugging Face**
Hugging Face is seeking passionate individuals to join our team. We believe in empowering our employees, providing opportunities for growth, and nurturing a culture of continuous learning.

**Current Openings:**
- Software Engineers
- Data Scientists
- Product Managers
- AI Researchers

Join us as we innovate and build the next generation of AI tools!

---

## **Get Involved**
To learn more about our offerings or to join our community, visit our website at [Hugging Face](https://huggingface.co).

**Follow Us on Social Media:**
- [GitHub](https://github.com/huggingface)
- [Twitter](https://twitter.com/huggingface)
- [LinkedIn](https://www.linkedin.com/company/huggingface)

Let’s build the future of AI together!

---

In [24]:
# Try changing the system prompt to the humorous version when you make the Brochure for Hugging Face:

stream_brochure("TechMahindra", "https://techmahindra.com")

Found links: {'links': [{'type': 'about page', 'url': 'https://techmahindra.com/about-us/'}, {'type': 'leadership page', 'url': 'https://techmahindra.com/about-us/leadership/'}, {'type': 'our brand page', 'url': 'https://techmahindra.com/about-us/our-brand/'}, {'type': 'sustainability page', 'url': 'https://techmahindra.com/about-us/sustainability/'}, {'type': 'corporate citizenship page', 'url': 'https://techmahindra.com/about-us/corporate-citizenship/'}, {'type': 'careers page', 'url': 'https://careers.techmahindra.com/'}, {'type': 'techm way page', 'url': 'https://techmahindra.com/careers/techm-way/'}, {'type': 'diversity and inclusion page', 'url': 'https://techmahindra.com/careers/diversity-inclusion/'}, {'type': 'awards page', 'url': 'https://techmahindra.com/awards/'}, {'type': 'news page', 'url': 'https://techmahindra.com/insights/news/'}]}


# Tech Mahindra Company Brochure

---

### **Company Overview**
Tech Mahindra is a leading provider of digital transformation, consulting, and business re-engineering services and solutions. Our promise, "Scale at Speed™", encapsulates our commitment to helping enterprises across industries transition at an accelerated pace to ensure agility, resilience, and efficiency in their businesses.

### **Core Services**
We offer a diverse range of services including:

- **Consulting Services**
- **Application Services** 
- **Intelligent Automation**
- **Cloud & Infrastructure Services**
- **Data Analytics**
- **Artificial Intelligence**
- **Cyber Security**
- **Blockchain**
- **Metaverse Solutions**

### **Industries Served**
Our expertise spans over 13 industries, including:

- Banking & Financial Services
- Healthcare & Life Sciences
- Energy & Utilities
- Media & Entertainment
- Manufacturing
- Retail & Consumer Goods

### **Commitment to Sustainability**
We prioritize maintaining a balance between profitability and sustainability. Recognized as a global sustainability leader by S&P Dow Jones Sustainability Indices, we implement innovative processes that not only provide value to our customers but also contribute to a sustainable future.

### **Company Culture**
At Tech Mahindra, our core philosophy is encapsulated in **"Rise"**. We aim for an equal world, prepare our teams for the future, and create lasting value. Diversity and inclusion are keystones of our corporate culture, fostering an environment where every employee is empowered to thrive and contribute.

- **The TechM Way**: Focused on collaboration, innovation, and continuous improvement.
- **Recognition**: Celebrated for our commitment to excellence and industry-leading best practices.

### **Careers at Tech Mahindra**
Joining our team means becoming part of a dynamic work environment that values innovation and personal growth. We encourage prospective candidates to explore opportunities that empower them to make a difference. 

- **Why Join Us?**
  - Opportunity to work on cutting-edge technologies.
  - Comprehensive learning and development programs.
  - Commitment to work-life balance.

Explore career opportunities at [Tech Mahindra Careers](#).

### **Contact Us**
For more information about our solutions, career opportunities, or partnerships, please feel free to reach out:

- **Website**: [techmahindra.com](http://techmahindra.com)
- **Email**: [contact@techmahindra.com](mailto:contact@techmahindra.com)

---

Join us at Tech Mahindra to transform industries with unparalleled speed and innovation!

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../business.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#181;">Business applications</h2>
            <span style="color:#181;">In this exercise we extended the Day 1 code to make multiple LLM calls, and generate a document.

This is perhaps the first example of Agentic AI design patterns, as we combined multiple calls to LLMs. This will feature more in Week 2, and then we will return to Agentic AI in a big way in Week 8 when we build a fully autonomous Agent solution.

Generating content in this way is one of the very most common Use Cases. As with summarization, this can be applied to any business vertical. Write marketing content, generate a product tutorial from a spec, create personalized email content, and so much more. Explore how you can apply content generation to your business, and try making yourself a proof-of-concept prototype. See what other students have done in the community-contributions folder -- so many valuable projects -- it's wild!</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../important.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#900;">Before you move to Week 2 (which is tons of fun)</h2>
            <span style="color:#900;">Please see the week1 EXERCISE notebook for your challenge for the end of week 1. This will give you some essential practice working with Frontier APIs, and prepare you well for Week 2.</span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../resources.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#f71;">A reminder on 3 useful resources</h2>
            <span style="color:#f71;">1. The resources for the course are available <a href="https://edwarddonner.com/2024/11/13/llm-engineering-resources/">here.</a><br/>
            2. I'm on LinkedIn <a href="https://www.linkedin.com/in/eddonner/">here</a> and I love connecting with people taking the course!<br/>
            3. I'm trying out X/Twitter and I'm at <a href="https://x.com/edwarddonner">@edwarddonner<a> and hoping people will teach me how it's done..  
            </span>
        </td>
    </tr>
</table>

<table style="margin: 0; text-align: left;">
    <tr>
        <td style="width: 150px; height: 150px; vertical-align: middle;">
            <img src="../thankyou.jpg" width="150" height="150" style="display: block;" />
        </td>
        <td>
            <h2 style="color:#090;">Finally! I have a special request for you</h2>
            <span style="color:#090;">
                My editor tells me that it makes a MASSIVE difference when students rate this course on Udemy - it's one of the main ways that Udemy decides whether to show it to others. If you're able to take a minute to rate this, I'd be so very grateful! And regardless - always please reach out to me at ed@edwarddonner.com if I can help at any point.
            </span>
        </td>
    </tr>
</table>